In [5]:
import os
import pickle
DATA_PATH = os.environ.get('dataset_filepath')
BUGGY_DATAPATH = DATA_PATH+"/buggy_dataset"

In [8]:
for file in os.listdir(BUGGY_DATAPATH):
    filename = os.fsdecode(file)
    if filename.endswith(".pickle"):
        print("found pickle file")
        f =  open(BUGGY_DATAPATH+"/"+filename,"rb")
        data = pickle.load(f)
        print(data)

found pickle file


ModuleNotFoundError: No module named 'pandas.core.indexes.numeric'